# 📊 GDP Per Capita Data Exploration & Cleaning

In this notebook, we will explore and clean the World Bank GDP per capita dataset.

## 🎯 Objectives:
- Understand the structure of the dataset
- Analyze missing values
- Basic statistical properties
- Data cleaning procedures

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries loaded successfully!")

In [ ]:
# Load the dataset (using relative path - suitable for GitHub)
df = pd.read_csv('../data/gdp-per-capita-worldbank.csv')

print(f"📈 Dataset loaded successfully!")
print(f"📊 Dimensions: {df.shape[0]} rows x {df.shape[1]} columns")

## 🔍 Dataset Structure Analysis

In [ ]:
# Display first 5 rows
print("📋 First 5 rows:")
display(df.head())

In [ ]:
# Column information
print("📄 Column Information:")
print(df.info())

In [ ]:
# Check column names
print("📝 Column Names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

In [ ]:
# Number of unique countries
print(f"🌍 Total number of countries/regions: {df['Entity'].nunique()}")
print(f"📅 Year range: {df['Year'].min()} - {df['Year'].max()}")
print(f"📊 Total data points: {len(df)}")

## 🔎 Missing Values Analysis

In [ ]:
# Missing values check
print("❌ Missing Values Analysis:")
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})

display(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Use the actual GDP column name
gdp_column = [col for col in df.columns if 'GDP' in col or 'capita' in col or col not in ['Entity', 'Code', 'Year']][0]
print(f"💰 GDP column: {gdp_column}")

# Countries with missing GDP values
missing_gdp = df[df[gdp_column].isnull()]
print(f"\n❌ Number of records without GDP data: {len(missing_gdp)}")

if len(missing_gdp) > 0:
    print("\n🔍 Some countries with missing GDP data:")
    print(missing_gdp[['Entity', 'Year']].head(10))

## 📊 Basic Statistical Analysis

In [ ]:
# Basic statistics
print("📈 GDP Per Capita Basic Statistics:")
display(df[gdp_column].describe())

In [ ]:
# Visualize GDP distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df[gdp_column].dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('GDP Per Capita Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('GDP Per Capita (USD)')
axes[0].set_ylabel('Frequency')
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(df[gdp_column].dropna(), vert=True)
axes[1].set_title('GDP Per Capita Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('GDP Per Capita (USD)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Outlier analysis
Q1 = df[gdp_column].quantile(0.25)
Q3 = df[gdp_column].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df[gdp_column] < lower_bound) | (df[gdp_column] > upper_bound)]
print(f"\n🔍 Number of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")

## 🌍 Countries and Years Analysis

In [ ]:
# Countries with most and least data points
country_data_count = df.groupby('Entity').size().sort_values(ascending=False)

print("🏆 Countries with most data points:")
print(country_data_count.head(10))

print("\n⚠️ Countries with least data points:")
print(country_data_count.tail(10))

In [ ]:
# Number of data points by year
yearly_count = df.groupby('Year').size()

plt.figure(figsize=(12, 6))
yearly_count.plot(kind='line', marker='o', linewidth=2, markersize=4)
plt.title('Number of Data Points by Year', fontsize=14, fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Number of Data Points')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"📊 Year with most data: {yearly_count.idxmax()} ({yearly_count.max()} countries)")
print(f"📊 Year with least data: {yearly_count.idxmin()} ({yearly_count.min()} countries)")

## 🧹 Data Cleaning

In [ ]:
# Data size before cleaning
print(f"🔢 Data size before cleaning: {df.shape}")

# 1. Remove missing GDP values
df_clean = df.dropna(subset=[gdp_column]).copy()
print(f"✅ Missing GDP values removed: {df_clean.shape}")

# 2. Check for negative GDP values
negative_gdp = df_clean[df_clean[gdp_column] < 0]
if len(negative_gdp) > 0:
    print(f"⚠️ Records with negative GDP: {len(negative_gdp)}")
    df_clean = df_clean[df_clean[gdp_column] >= 0]
    print(f"✅ Negative values removed: {df_clean.shape}")
else:
    print("✅ No negative GDP values found")

# 3. Check for extreme outliers (very high values)
extreme_outliers = df_clean[df_clean[gdp_column] > 200000]  # Above 200k USD
if len(extreme_outliers) > 0:
    print(f"\n🔍 Extremely high GDP values ({len(extreme_outliers)} records):")
    print(extreme_outliers[['Entity', 'Year', gdp_column]].head())

print(f"\n🎉 Final cleaned data size: {df_clean.shape}")
print(f"📉 Percentage of removed records: {((len(df) - len(df_clean)) / len(df) * 100):.1f}%")

In [ ]:
# Summary of cleaned data
print("📊 Cleaned Data Summary:")
print(f"🌍 Number of countries: {df_clean['Entity'].nunique()}")
print(f"📅 Year range: {df_clean['Year'].min()} - {df_clean['Year'].max()}")
print(f"💰 GDP range: ${df_clean[gdp_column].min():,.0f} - ${df_clean[gdp_column].max():,.0f}")
print(f"📈 Average GDP: ${df_clean[gdp_column].mean():,.0f}")
print(f"📊 Median GDP: ${df_clean[gdp_column].median():,.0f}")

In [ ]:
# Create cleaned dataset by removing missing GDP values
df_clean = df.dropna(subset=[gdp_column]).copy()

# Remove negative GDP values if any
df_clean = df_clean[df_clean[gdp_column] >= 0]

print(f"📊 Cleaned dataset size: {df_clean.shape}")
print(f"📈 Average GDP: ${df_clean[gdp_column].mean():,.0f}")
print(f"📊 Median GDP: ${df_clean[gdp_column].median():,.0f}")

# Save cleaned dataset
df_clean.to_csv('../data/gdp_cleaned.csv', index=False)
print("💾 Cleaned data saved as '../data/gdp_cleaned.csv'!")

# Set global variable for usage
GDP_COLUMN = gdp_column
print(f"\n🔧 GDP column name: {GDP_COLUMN}")
print("✅ Data exploration and cleaning completed!")